#### 목표변수와 설명변수
- Subject: 환자
- Goal: Features가 Target인 수술실패여부에 미치는 영향 분석
- Target: '수술실패여부'
- Features: '고혈압여부', '성별', '신부전여부', '연령', '체중', '수술시간'

In [2]:
import pandas as pd

In [3]:
df_ROS = pd.read_csv('../datasets/RecurrenceOfSurgery.csv')
df_ROS_select = df_ROS[['수술실패여부', '고혈압여부', '성별', '신부전여부', '연령', '체중', '수술시간']]
df_ROS_select[:2]

,수술실패여부,고혈압여부,성별,신부전여부,연령,체중,수술시간
0,0,0,2,0,66,60.3,68.0
1,0,0,1,0,47,71.7,31.0


In [4]:
df_ROS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              1894 non-null   int64  
 1   환자ID                    1894 non-null   object 
 2   Large Lymphocyte        1894 non-null   float64
 3   Location of herniation  1894 non-null   int64  
 4   ODI                     462 non-null    float64
 5   가족력                     1843 non-null   float64
 6   간질성폐질환                  1894 non-null   int64  
 7   고혈압여부                   1894 non-null   int64  
 8   과거수술횟수                  1894 non-null   int64  
 9   당뇨여부                    1894 non-null   int64  
 10  말초동맥질환여부                1894 non-null   int64  
 11  빈혈여부                    1894 non-null   int64  
 12  성별                      1894 non-null   int64  
 13  스테로이드치료                 1894 non-null   int64  
 14  신부전여부                   1894 non-null   

In [5]:
df_ROS_select.isnull().sum()

수술실패여부     0
고혈압여부      0
성별         0
신부전여부      0
연령         0
체중         0
수술시간      54
dtype: int64

In [6]:
# 수술시간이 Null인 행 삭제
df_ROS_select = df_ROS_select.dropna(subset=['수술시간'], how='any', axis=0)

In [7]:
df_ROS_select.isnull().sum()

수술실패여부    0
고혈압여부     0
성별        0
신부전여부     0
연령        0
체중        0
수술시간      0
dtype: int64

#### 정형화

In [8]:
target = df_ROS_select['수술실패여부']
features = df_ROS_select[['연령', '체중', '수술시간']]
target.shape, features.shape

((1840,), (1840, 3))

In [9]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.3, random_state = 10)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((1288, 3), (552, 3), (1288,), (552,))

#### 모델학습

In [10]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
model = DecisionTreeClassifier()
# Target이 범주형이므로 Classifier을 사용

In [11]:
from sklearn.model_selection import GridSearchCV
hyper_params = {'min_samples_leaf' : range(2,5),
               'max_depth' : range(2,5),
               'min_samples_split' : range(2,5)}

#### 평가 Score Default, 분류(Accuracy), 예측(R square)

In [12]:
from sklearn.metrics import f1_score, make_scorer
scoring = make_scorer(f1_score)

In [13]:
grid_search = GridSearchCV(model, param_grid = hyper_params, cv=2, verbose=1, scoring=scoring)
grid_search.fit(features, target)

Fitting 2 folds for each of 27 candidates, totalling 54 fits


GridSearchCV(cv=2, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(2, 5),
                         'min_samples_leaf': range(2, 5),
                         'min_samples_split': range(2, 5)},
             scoring=make_scorer(f1_score), verbose=1)

In [14]:
grid_search.best_estimator_

DecisionTreeClassifier(max_depth=3, min_samples_leaf=2)

In [20]:
grid_search.best_score_, grid_search.best_params_

# 전처리 전의 정확도(accuracy) : 0.028571428571428574
# 낮은 정확도, 모델이 예측을 잘 수행하지 못함

(0.028571428571428574,
 {'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 2})

In [16]:
best_model = grid_search.best_estimator_
best_model

DecisionTreeClassifier(max_depth=3, min_samples_leaf=2)

In [17]:
target_test_predict = best_model.predict(features_test)
target_test_predict

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [18]:
from sklearn.metrics import classification_report

In [19]:
print(classification_report(target_test, target_test_predict))

# 전처리 전의 값
#  precision  recall  f1-score   support
#0  0.94      1.00      0.97       516
#1  1.00      0.03      0.05        36

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       516
           1       1.00      0.03      0.05        36

    accuracy                           0.94       552
   macro avg       0.97      0.51      0.51       552
weighted avg       0.94      0.94      0.91       552



### 결론
- 의학 통계에서는 실제 instances들 중 해당하는 것을 identify하는 정도를 측정하는 recall이 더 중요함